<a href="https://colab.research.google.com/github/objectc/CNN-with-TensorFlow2.0-and-Keras/blob/master/Resnet_residual_block.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#import needed classes
from tensorflow import keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,AveragePooling2D,Dropout,BatchNormalization,Activation, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import ModelCheckpoint
from math import ceil
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [0]:
def Unit(x,filters,pool=False):
    res = x
    if pool:
        x = MaxPooling2D(pool_size=(2, 2))(x)
        res = Conv2D(filters=filters,kernel_size=[1,1],strides=(2,2),padding="same")(res)
    out = BatchNormalization()(x)
    out = Activation("relu")(out)
    out = Conv2D(filters=filters, kernel_size=[3, 3], strides=[1, 1], padding="same")(out)

    out = BatchNormalization()(out)
    out = Activation("relu")(out)
    out = Conv2D(filters=filters, kernel_size=[3, 3], strides=[1, 1], padding="same")(out)

    out = keras.layers.add([res,out])

    return out


In [0]:
#Define the model


def MiniModel(input_shape):
    images = Input(input_shape)
    net = Conv2D(filters=32, kernel_size=[3, 3], strides=[1, 1], padding="same")(images)
    net = Unit(net,32)
    net = Unit(net,32)
    net = Unit(net,32)

    net = Unit(net,64,pool=True)
    net = Unit(net,64)
    net = Unit(net,64)

    net = Unit(net,128,pool=True)
    net = Unit(net,128)
    net = Unit(net,128)
    net = Unit(net, 256,pool=True)
    net = Unit(net, 256)
    net = Unit(net, 256)

    net = BatchNormalization()(net)
    net = Activation("relu")(net)
    net = Dropout(0.25)(net)

    net = AveragePooling2D(pool_size=(4,4))(net)
    net = Flatten()(net)
    net = Dense(units=10,activation="softmax")(net)

    model = Model(inputs=images,outputs=net)

    return model

In [15]:
#load the cifar10 dataset
(train_x, train_y) , (test_x, test_y) = cifar10.load_data()

#normalize the data
train_x = train_x/255
test_x = test_x/255
#Subtract the mean image from both train and test set
# train_x = train_x - train_x.mean()
# test_x = test_x - test_x.mean()

#Divide by the standard deviation
# train_x = train_x / train_x.std(axis=0)
# test_x = test_x / test_x.std(axis=0)


datagen = ImageDataGenerator(rotation_range=10,
                             width_shift_range=5. / 32,
                             height_shift_range=5. / 32,
                             rescale=1./255,
                             horizontal_flip=True)

# Compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(train_x)



#Encode the labels to vectors
train_y = keras.utils.to_categorical(train_y,10)
test_y = keras.utils.to_categorical(test_y,10)

#define a common unit


input_shape = (32,32,3)
model = MiniModel(input_shape)

#Print a Summary of the model

model.summary()
#Specify the training components
model.compile(optimizer=Adam(0.001),loss="categorical_crossentropy",metrics=["accuracy"])

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_56 (Conv2D)              (None, 32, 32, 32)   896         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_50 (BatchNo (None, 32, 32, 32)   128         conv2d_56[0][0]                  
__________________________________________________________________________________________________
activation_50 (Activation)      (None, 32, 32, 32)   0           batch_normalization_50[0][0]     
____________________________________________________________________________________________

In [19]:
epochs = 50
steps_per_epoch = ceil(50000/128)

# Fit the model on the batches generated by datagen.flow().
model.fit(datagen.flow(train_x, train_y, batch_size=128),
                    validation_data=(test_x,test_y),
                    epochs=epochs,
                    steps_per_epoch=steps_per_epoch, 
                    verbose=1
                   )


#Evaluate the accuracy of the test dataset
accuracy = model.evaluate(x=test_x,y=test_y,batch_size=128)
model.save("cifar10model.h5")

Epoch 1/50
391/391 [==============================] - 47s 119ms/step - loss: 0.7974 - accuracy: 0.7197 - val_loss: 1.0746 - val_accuracy: 0.6370
Epoch 2/50
391/391 [==============================] - 46s 117ms/step - loss: 0.6756 - accuracy: 0.7651 - val_loss: 0.7172 - val_accuracy: 0.7643
Epoch 3/50
391/391 [==============================] - 46s 118ms/step - loss: 0.5920 - accuracy: 0.7939 - val_loss: 0.8542 - val_accuracy: 0.7266
Epoch 4/50
391/391 [==============================] - 46s 118ms/step - loss: 0.5348 - accuracy: 0.8128 - val_loss: 0.7105 - val_accuracy: 0.7519
Epoch 5/50
391/391 [==============================] - 46s 118ms/step - loss: 0.4935 - accuracy: 0.8263 - val_loss: 0.6186 - val_accuracy: 0.7840
Epoch 6/50
391/391 [==============================] - 46s 118ms/step - loss: 0.4588 - accuracy: 0.8391 - val_loss: 0.6526 - val_accuracy: 0.7835
Epoch 7/50
391/391 [==============================] - 46s 118ms/step - loss: 0.4263 - accuracy: 0.8505 - val_loss: 0.7532 - val_ac